In [11]:
import numpy as np
from regain.data.base import load_Petrobras
from regain.hmm.higher_order_hmm_graphical_lasso import HMM_GraphicalLasso
from regain.hmm.utils import corr_plot,plot_results_cluster,cov2corr,cluster_returns_recap
import matplotlib.pyplot as plt
from decimal import Decimal
import pandas as pd

## Load data

In [ ]:
Petrob, BrazCurr,WTI = load_Petrobras()


In [ ]:
ind_WTI = np.zeros(np.size(WTI.index),bool)
for i,date in enumerate(WTI.index):
    ind_WTI[i] = np.any(Petrob.index==date)


New_Dates = WTI.index[ind_WTI]


ind_braz = np.zeros(np.size(BrazCurr.index),bool)
for i,date in enumerate(BrazCurr.index):
    ind_braz[i] = np.any(New_Dates==date)
 
ind_Petr = np.zeros(np.size(Petrob.index),bool)
for i,date in enumerate(Petrob.index):
    ind_Petr[i] = np.any(New_Dates==date)


In [ ]:
BrazCurr_filt = BrazCurr['Ultimo'].values[ind_braz]
WTI_filt = WTI['Ultimo'].values[ind_WTI]
Petrob_filt = Petrob['Ultimo'].values[ind_Petr]
data = np.zeros((np.size(BrazCurr_filt),3))

for i in range(np.size(BrazCurr_filt)):
    data[i,0] = float(BrazCurr_filt[i].replace(',','.'))
    data[i,1] = float(Petrob_filt[i].replace(',','.'))
    data[i,2] = float(WTI_filt[i].replace(',','.'))


## Compute returns

In [ ]:

returns_mat = np.zeros((np.size(data,axis = 0)-1,np.size(data,axis = 1)))

for i in range(1,np.size(data,axis = 0)):
    returns_mat[i-1,:] = (data[i,:]-data[i-1,:])/data[i-1,:]*100
returns_mat = np.flip(returns_mat,axis=0)

plt.plot(returns_mat)
plt.show()

## Plot data

In [ ]:
import matplotlib.pyplot as plt


def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)


fig, host = plt.subplots()
fig.subplots_adjust(right=0.75)

par1 = host.twinx()
par2 = host.twinx()

# Offset the right spine of par2.  The ticks and label have already been
# placed on the right by twinx above.
par2.spines["right"].set_position(("axes", 1.2))
# Having been created by twinx, par2 has its frame off, so the line of its
# detached spine is invisible.  First, activate the frame but make the patch
# and spines invisible.
make_patch_spines_invisible(par2)
# Second, show the right spine.
par2.spines["right"].set_visible(True)

p1, = host.plot(data[:,0], "b-", label="Braz_Curr")
p2, = par1.plot(data[:,1], "r-", label="Petrobras")
p3, = par2.plot(data[:,2], "g-", label="WTI")


host.set_ylabel("Braz_Curr")
par1.set_ylabel("Petrobras")
par2.set_ylabel("WTI")

host.yaxis.label.set_color(p1.get_color())
par1.yaxis.label.set_color(p2.get_color())
par2.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)
host.tick_params(axis='y', colors=p1.get_color(), **tkw)
par1.tick_params(axis='y', colors=p2.get_color(), **tkw)
par2.tick_params(axis='y', colors=p3.get_color(), **tkw)
host.tick_params(axis='x', **tkw)

lines = [p1, p2, p3]

host.legend(lines, [l.get_label() for l in lines])

plt.show()

# Investment results

In [ ]:
from regain.hmm.utils_portfolio_optimization import PO_with_HMM_GMM
N_backtest = 1000
Invest_wealth = 1e5


res_pred,res_today,res_emp = PO_with_HMM_GMM(returns_mat,
                                     data[1:,:],
                                     np.linspace(0,50,3),
                                     [2,3],
                                     N_max_mem = 1500,
                                     N_test=N_backtest,
                                     Wealth=Invest_wealth,
                                     mu_p= 0.1,
                                     OP_method = 'Min_Var_Fix_return',
                                     leverage=True)

In [ ]:
rcParams.update({'font.size':15})

fig3 = plt.figure(constrained_layout=True, figsize=(15,8))
gs = fig3.add_gridspec(5, 2)
f3_ax1 = fig3.add_subplot(gs[0:2, :-1])
f3_ax2 = fig3.add_subplot(gs[2, :-1])
f3_ax3 = fig3.add_subplot(gs[3, :-1])
f3_ax4 = fig3.add_subplot(gs[4, :-1])
f3_ax5 = fig3.add_subplot(gs[:, 1])

f3_ax1.plot(dates,data[-2500:-865,0]/np.max(data[-2500:-865,0]), "b-", label="USD/BRL")
f3_ax1.plot(dates,data[-2500:-865,1]/np.max(data[-2500:-865,1]), "r-", label="PETR4")
f3_ax1.plot(dates,data[-2500:-865,2]/np.max(data[-2500:-865,2]), "g-", label="WTI Crude Oil")

f3_ax1.set_xticklabels([])
f3_ax1.legend(ncol=3, bbox_to_anchor=(0.067, 1.005), fontsize=14)

f3_ax1.set_ylabel('Normalized prices')



f3_ax2.plot(dates, [(data[i,0] - data[i-1, 0])/data[i-1, 0]*100 for i in range(-2500, -865)], "b-")
f3_ax3.plot(dates, [(data[i,1] - data[i-1, 1])/data[i-1, 1]*100 for i in range(-2500, -865)], "r-")
f3_ax4.plot(dates, [(data[i,2] - data[i-1, 2])/data[i-1, 2]*100 for i in range(-2500, -865)], "g-")

f3_ax2.set_yticks([-5, 0, 5])
f3_ax2.set_yticklabels(['  -5 %', '0 %', '  5 %'])

f3_ax2.set_xticklabels([])
f3_ax2.set_ylabel('Variation')



f3_ax3.set_yticks([-10, 0, 10])
f3_ax3.set_yticklabels(['-10 %', '0 %', '10 %'])


f3_ax3.set_xticklabels([])

f3_ax3.set_ylabel('Variation')

f3_ax4.set_yticks([-10, 0, 10])
f3_ax4.set_yticklabels(['-10 %', '0 %', '10 %'])

f3_ax4.set_ylabel('Variation')
#f3_ax4.set_xlabel('Dates')

# lines = [p1, p2, p3]
f3_ax5.plot(dates[-165:],np.cumsum(res_pred)/1000, lw=3, color='C0', label='HMM-GGM')
f3_ax5.plot(dates[-165:],np.cumsum(res_emp)/1000, lw=3, color='C3',label='Emp_cov(50 days)')
f3_ax5.legend(ncol=2, bbox_to_anchor=(0.9, 1.1), fontsize=14)
f3_ax5.set_xticklabels(dates[-165:], rotation=90)
#f3_ax5.set_xlabel('Dates')




degrees = 90
#f3_ax5.set_xticks(rotation=degrees)
f3_ax5.yaxis.tick_right()
f3_ax5.yaxis.set_label_position("right")
f3_ax5.set_ylabel('P&L')
f3_ax5.set_yticklabels([str(int(t))+' %' for t in f3_ax5.get_yticks()])
f3_ax5.grid()

plt.tight_layout()
#plt.ylabel("Percent of capital\n")
#host.legend(lines, [l.get_label() for l in lines],loc='lower right', bbox_to_anchor=(1, 0.5))
plt.savefig("investement_real_data.pdf", dpi=200, bbox_inches='tight', transparent=True)
#plt.show()

## Regression results

In [ ]:
from regain.hmm.utils_pred import reg_pred_HMM_GMM
N_pred = 30
Data = data[-2500:-865,:]
ret = returns_mat[-2500:-865,:]



ret_pred,_, Val_Pred =   reg_pred_HMM_GMM(ret,
                                   Data,
                                   [10,20,30,40,50],
                                   [2,3,4],
                                   N_retrain = N_pred,
                                   N_val = N_pred,
                                   p=2,
                                   N_test = N_pred,
                                   meth = 'viterbi',
                                   pred_meth = 'rolling',
                                   recrossval = False,
                                   CV_meth = 'reg',
                                   perc_var=True)

In [ ]:
from notebooks.hmm.pred_func import pred_regression_methods
methods = ['lgb','LSTM','VAR','Kernel_RBF']
res_regre = []

for meth in methods:

    _, Val_pred, _ = pred_regression_methods(Data,ret,
                                    N_test=N_pred,
                                    method = meth,
                                    N_val = N_pred,
                                    pred_meth = 'rolling',
                                    p = 2,
                                    plot=False,
                                    perc_var=True)
    res_regre.append(Val_pred)

In [ ]:
res_regre.append(Val_Pred)
methods.append('HHM_GGM2')

In [ ]:
figsizex=15
figsizey=20
N_per_rows=2
N_mem = 10
N_test = N_pred
Dates = None
columns = None
N_TS = np.size(Data, axis=1)


N_rows = int(np.ceil(N_TS / N_per_rows))
f, axes = plt.subplots(N_rows, N_per_rows, figsize=(figsizex, figsizey))
list_all = []
for ts in range(N_TS):
    i = int(ts / N_per_rows)
    j = np.remainder(ts, N_per_rows)
    
    for n,Value_pred in enumerate(res_regre):
        
        absolute_error = abs(Data[-(N_test):,ts]-Value_pred[:,ts])
        MAE = np.mean(absolute_error)
        std = np.std(absolute_error)
        

        if Dates is None:
            x_mem = np.arange(N_mem + N_test)
            x = np.arange(N_mem,N_mem + N_test)
            if n==0:
                axes[i, j].plot(x_mem, Data[-(N_mem+N_test):,ts],label='real')
                #axes[i, j].plot(x, Data[-(N_test):,ts], 'o', label='real')
            axes[i, j].plot(x, Value_pred[:,ts],  label='pred '+str(methods[n]))
            if columns is None:
                axes[i, j].set_title('var ' + str(ts) + ' forecast')
                list_all.append([str(methods[n]),'var ' + str(ts),MAE,std])
            else:
                axes[i, j].set_title(str(columns[ts]) + ' forecast')
                list_all.append([str(methods[n]),str(columns[ts]),MAE,std])

        axes[i, j].legend()

In [ ]:
df_recap = pd.DataFrame(list_all, columns=['method', 'TS', 'MAE', 'std AB'])

In [ ]:
df_recap.loc[df_recap['method'] == 'HHM_GGM2']['MAE'].mean()

In [ ]:
df_recap.loc[df_recap['method'] == 'VAR']['MAE'].mean()

In [ ]:
import pandas as pd
import numpy as np
res = pd.DataFrame(np.zeros((1, 5)), index=['MAE'], columns=np.unique(df_recap['method']))

for m in res.columns:
    res[m] = str(round(df_recap.loc[df_recap['method'] == m]['MAE'].mean(), 3)) + ' +/- '+ str(round(np.sqrt(np.mean(df_recap.loc[df_recap['method'] == m]['std AB']**2)), 3))
res.to_latex()

In [ ]:
df_recap.head()